In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import convert_all as crd

In [ ]:
# get all low_dim data (head&spine velocity control)
raw_dir = "../demonstrations/raw/mmk_pick_grape"
data = crd.raw_to_dict(
    raw_dir,
    ["low_dim.json"],
    video_file_names=None,
    flatten_mode="hdf5",
    concatenater={
        "/observations/qpos": (
            "/observation/arm/left/joint_position",
            "/observation/eef/left/joint_position",
            "/observation/arm/right/joint_position",
            "/observation/eef/right/joint_position",
            "/observation/head/joint_position",
            "/observation/spine/joint_position"
        ),
        "/action": (
            "/action/arm/left/joint_position",
            "/action/eef/left/joint_position",
            "/action/arm/right/joint_position",
            "/action/eef/right/joint_position",
            "/action/head/joint_velocity",
            "/action/spine/joint_velocity"
        ),
    },
    key_filter=[
        "/observation/arm/left/joint_velocity",
        "/observation/arm/right/joint_velocity",
        "/observation/arm/left/joint_torque",
        "/observation/arm/right/joint_torque",
    ],
)

In [ ]:
ep_number = len(data)
print(f"Number of episodes: {ep_number}")

In [ ]:
print(list(data["0"].keys()))
print(list(data["0"]["/observations/qpos"][0]))
print(list(data["0"]["/action"][0]))

In [ ]:
# 将spine的observation[-120000, 0]缩放到[-2pi, 0]
import numpy as np
for i in range(ep_number):
    qpos = np.array(data[str(i)]["/observations/qpos"])
    qpos[:, -1] = 2 * np.pi * qpos[:, -1] / 120000
    data[str(i)]["/observations/qpos"] = qpos.tolist()

In [ ]:
import os
import cv2
from concurrent.futures import ThreadPoolExecutor

# merge high_dim data and save
raw_dir
video_names = ["head.avi"]
target_dir = f"data/hdf5/mmk_pick_grape_pos/"
low_dim_data = data
name_converter = {"head": "/observations/images/0"}
target_namer = lambda i: f"episode_{i}.hdf5"

compresser = crd.Compresser("jpg", [int(cv2.IMWRITE_JPEG_QUALITY), 50], True)

os.makedirs(target_dir, exist_ok=True)

# get max episode length
episode_lens = []
for low_d in low_dim_data.values():
    episode_lens.append(len(list(low_d.values())[0]))
max_pad_lenth = max(episode_lens)

episode_names = list(low_dim_data.keys())
print(f"Max episode length: {max_pad_lenth}")
print(f"All episodes: {episode_names}")
print(f"episode number: {len(episode_names)}")

In [ ]:
# save all data
futures = []
with ThreadPoolExecutor(max_workers=25) as executor:

    def save_one(index, ep_name):
        crd.merge_video_and_save(
            low_dim_data[ep_name],
            f"{raw_dir}/{ep_name}",
            video_names,
            crd.save_dict_to_hdf5,
            name_converter,
            compresser,
            f"{target_dir}/" + target_namer(index),
            max_pad_lenth,
        )
        data.pop(ep_name)

    for index, ep_name in enumerate(episode_names):
        futures.append(executor.submit(save_one, index, ep_name))

In [ ]:
# fix action
import json
import numpy as np

for i in range(49, 50):
    with open(f"demonstrations/raw/mmk_pick_grape/{i}/low_dim.json", "r") as f:
        data = json.load(f)
        action = data["action"]["spine"]["joint_position"]
        obs = data["observation"]["spine"]["joint_position"]
        # change action
        delta = np.array(obs[1:]) - np.array(obs[:-1])
        # > 0 -> 1, < 0 -> -1
        action = np.sign(delta)
        # 获取不为0的值的index
        index_not0 = np.where(action != 0)
        first_not0 = index_not0[0][0]
        last_not0 = index_not0[0][-1]
        # 从第一个不为0的值开始，到最后一个不为0的值结束，将中间的值设为1
        action[first_not0:last_not0] = 1
        action = action.tolist()
        action.append(action[-1])
        data["action"]["spine"] = {"joint_velocity": action}
    with open(f"demonstrations/raw/mmk_pick_grape/{i}/low_dim.json", "w") as f:
        json.dump(data, f)

In [ ]:
# rename
import json

for i in range(50):
    with open(f"demonstrations/raw/mmk_pick_grape/{i}/low_dim.json", "r") as f:
        data = json.load(f)
        data["action"]["head"] = {"joint_velocity": data["action"]["head"]["joint_velocity"]["joint_position"]}
    with open(f"demonstrations/raw/mmk_pick_grape/{i}/low_dim.json", "w") as f:
        json.dump(data, f)

## Position Control

In [ ]:
# add head and spine to joint_position action
import json

for i in range(58):
    with open(f"demonstrations/raw/mmk_pick_grape/{i}/low_dim.json", "r") as f:
        data = json.load(f)
        obs_spine = data["observation"]["spine"]["joint_position"]
        obs_head = data["observation"]["head"]["joint_position"]
        data["action"]["head"]["joint_position"] = obs_head[1:] + [obs_head[-1]]
        data["action"]["spine"]["joint_position"] = obs_spine[1:] + [obs_spine[-1]]
    with open(f"demonstrations/raw/mmk_pick_grape/{i}/low_dim.json", "w") as f:
        json.dump(data, f)

In [ ]:
# get all low_dim data (head&spine position control)
import convert_all as crd
raw_dir = "demonstrations/raw/mmk_pick_grape"
data = crd.raw_to_dict(
    raw_dir,
    ["low_dim.json"],
    video_file_names=None,
    flatten_mode="hdf5",
    concatenater={
        "/observations/qpos": (
            "/observation/arm/left/joint_position",
            "/observation/eef/left/joint_position",
            "/observation/arm/right/joint_position",
            "/observation/eef/right/joint_position",
            "/observation/head/joint_position",
            "/observation/spine/joint_position"
        ),
        "/action": (
            "/action/arm/left/joint_position",
            "/action/eef/left/joint_position",
            "/action/arm/right/joint_position",
            "/action/eef/right/joint_position",
            "/action/head/joint_position",
            "/action/spine/joint_position"
        ),
    },
    key_filter=[
        "/observation/arm/left/joint_velocity",
        "/observation/arm/right/joint_velocity",
        "/observation/arm/left/joint_torque",
        "/observation/arm/right/joint_torque",
        "/action/head/joint_velocity",
        "/action/spine/joint_velocity"
    ],
)

In [ ]:
ep_number = len(data)
print(f"Number of episodes: {ep_number}")

In [ ]:
# 将spine的observation和action[-120000, 0]缩放到[-2pi, 0]
import numpy as np
for i in range(ep_number):
    qpos_obs = np.array(data[str(i)]["/observations/qpos"])
    qpos_act = np.array(data[str(i)]["/action"])
    qpos_obs[:, -1] = 2 * np.pi * qpos_obs[:, -1] / 120000
    qpos_act[:, -1] = 2 * np.pi * qpos_act[:, -1] / 120000
    data[str(i)]["/observations/qpos"] = qpos_obs.tolist()
    data[str(i)]["/action"] = qpos_act.tolist()